# Heasoftpy testing notebook

This notebook lays out how to verify the functionality in the requirement document here:

https://docs.google.com/document/d/19yX41TdxZRJ037nqk2qJJnyel_SAKqrH1a7koXQ8nU0/edit   


## Import and help

In [1]:
import os, shutil, sys

In [2]:
sys.path.insert(0,"/Users/tjaffe/space/sw/heasoft/")
import heasoftpy as hsp

<font color=red>This is still necessary. (If not set, then the prompting below works but the tool errors off complaining about the tty.  If it's set, the prompting works (?) and then so does the tool.)  Cannot this be done on import?</font>

In [3]:
## This should then somehow be done internally in heasoftpy, right?  Otherwise 
##  none of the below works for me.  If I don't do this, then I get:
##  
##  ERROR: Device not configured
##  Task ftverify 0.0 terminating with status 6
##  Unable to redirect prompts to the /dev/tty (at headas_stdio.c:152)
##  
os.environ["HEADASNOQUERY"]="True"

<font color=green>This correctly shows up the Docstring in the Jupyter Pager for Jupyter Notebook and here inline for JupyterLab.</font>

In [4]:
##   The docstring currently says:
##
##   from heaspy import *
##
##   and I think we talked about how this doesn't work and doesn't need to.
hsp?

<font color=red>**This works, but it isn't useful to the user.  The Description should include a quick usage guide**, e.g.,
    
    DESCRIPTION

        Python package to wrap the HEASoft tools so that they can be called from python scripts,
        interactive ipython sessions, or Jupyter Notebooks.  

        > import heasoftpy as hsp
        > hsp.fdump?
        ...
        > hsp.fdump()
        Name of FITS file and [ext#]> Crab_sum_pha.fits
        ...
        > result=hsp.fdump(infile='foo.fits',outfile='STDOUT',columns='RATE',rows='1-2',more='yes')
        > params2=result.params
        > params2['prhead']='no'
        etc. 

        or

          try:
            result=hsp.ftverify(params)
            print("Success")
          except HeasoftpyExecutionError as e:
            print("Error trapped as HeasoftpyExecutionError with message:{}".format(e))
          except Exception as e:
            print("Unexpected status--{}--".format(e))
    
        Version ...


In [4]:
help(hsp)

Help on package heasoftpy:

NAME
    heasoftpy - Creates a Python interface to the FTools/HTools package.

DESCRIPTION
    If there is not already a file containing a function to run a given FTools program,
    this will create one upon import. It uses the appropriate paramter file to do this.
    
    Version 0.1 ME: initial version
    Version 0.1.1 MFC: corrects identification of prompting for required, missing parameters
    Version 0.1.2 ME: Added Error classes and handling errors when the underlying program
                      encounters an error.
    Version 0.1.3 ME: Added checks of function versions and replacement of functions that
                      are outdated.
    
    ME = Matt Elliott
    MFC = Mike Corcoran

PACKAGE CONTENTS
    clean_heasoftpy
    errors
    onthefly (package)
    result
    utils
    write_fns (package)

CLASSES
    builtins.tuple(builtins.object)
        HToolsParameter
    
    class HToolsParameter(builtins.tuple)
     |  HToolsParameter(name

In [5]:
hsp.help()

AttributeError: module 'heasoftpy' has no attribute 'help'

<font color=green>Prompting now works in a notebook</font>  **<font color=red>But it does not present the default values to accept, which I think it should.</font>**

<font color=green>This also shows up in the pager and shows the parameters.  </font>

In [5]:
hsp.fdump?

<font color=red>Still no help function.  The question mark gives you the parameters, good.  But what about the full fhelp?  This should just call fhelp.</font>

In [6]:
##  I think we want something like this to print the help for ftverify
hsp.ftverify.help()

AttributeError: 'function' object has no attribute 'help'

<font color=red>**BRoken for me**</font>

In [7]:
# This errors if you give a tool name either as the argument or at the prompt.  
hsp.fhelp()

Task name> ftverify


HeasoftpyExecutionError: 
Error running: fhelp
returncode: 2
output:
Can't load '/Users/tjaffe/space/sw/heasoft/heasoft-6.22.1/x86_64-apple-darwin17.4.0/lib/perl/auto/HEACORE/PIL/PIL.bundle' for module HEACORE::PIL: dlopen(/Users/tjaffe/space/sw/heasoft/heasoft-6.22.1/x86_64-apple-darwin17.4.0/lib/perl/auto/HEACORE/PIL/PIL.bundle, 1): Library not loaded: @rpath/libhdinit_2.7.dylib
  Referenced from: /Users/tjaffe/space/sw/heasoft/heasoft-6.22.1/x86_64-apple-darwin17.4.0/lib/perl/auto/HEACORE/PIL/PIL.bundle
  Reason: image not found at /System/Library/Perl/5.18/darwin-thread-multi-2level/DynaLoader.pm line 194.
 at /Users/tjaffe/space/sw/heasoft/heasoft-6.22.1/x86_64-apple-darwin17.4.0/bin/fhelp line 96.
Compilation failed in require at /Users/tjaffe/space/sw/heasoft/heasoft-6.22.1/x86_64-apple-darwin17.4.0/bin/fhelp line 96.
BEGIN failed--compilation aborted at /Users/tjaffe/space/sw/heasoft/heasoft-6.22.1/x86_64-apple-darwin17.4.0/bin/fhelp line 96.


## Running and results

<font color=green>Returned data object:</font>

In [17]:
#  By default, everything to STDOUT
result=hsp.fdump(infile='Crab_sum_pha.fits',outfile='STDOUT',columns='STAT_ERR',rows='1-2',more='n',prhead='no')
result

Result(returncode=0, stdout=' \n \n        STAT_ERR\n        count/s\n      1   9.468074056581779E-02\n      2   6.595265185380932E-02\n', stderr=None, params={'infile': 'Crab_sum_pha.fits', 'outfile': 'STDOUT', 'columns': 'STAT_ERR', 'rows': '1-2', 'more': 'n', 'prhead': 'no'}, custom=None)

Seeing what's happening with error handling.  This example should produce errors.  

<font color=green>Error successfully trapped as an execution error from the tool</font>

In [28]:
from heasoftpy.errors import HeasoftpyExecutionError
try:
    result=hsp.fdump(infile='foo.fits',outfile='STDOUT',columns='-',rows='-',more='y')
    result
except HeasoftpyExecutionError as e:
    print("Error successfully trapped as HeasoftpyExecutionError with message:{}".format(e))
except Exception as e:
    print("Unexpected status--{}--".format(e))

Error successfully trapped as HeasoftpyExecutionError with message:
Error running: fdump
returncode: 1
output:
fdump3.5 : unable to open infile



Ftverify is an interesting case.  It can print things to STDERR but usually returns a good error status anyway.  How is this handled in heasofpy?

Split errors to stderr instead

In [41]:
## Tell it to separate out STDERR.  
if os.path.exists("ftverify.out"):
    os.unlink("ftverify.out")
result=hsp.ftverify(
    infile='Crab_sum_pha.fits',
    numerrs=10,numwrns=10,
    outfile="ftverify.out")
print(result.stdout)
print("--")
print(result.stderr)

*** Error:   Keyword #37, DATE-END: (from CFITSIO error stack:)
             input date string has illegal format:
             UTC_format
             
*** Error:   Keyword #36, DATE-OBS: (from CFITSIO error stack:)
             input date string has illegal format:
             UTC_format
             

--
None


In [40]:
## Tell it to separate out STDERR.  
if os.path.exists("ftverify.out"):
    os.unlink("ftverify.out")
result=hsp.ftverify(
    infile='Crab_sum_pha.fits',
    numerrs=10,numwrns=10,
    outfile="ftverify.out",
    stderr=True)
print(result.stdout)
print("--")
print(result.stderr)


--
*** Error:   Keyword #37, DATE-END: (from CFITSIO error stack:)
             input date string has illegal format:
             UTC_format
             
*** Error:   Keyword #36, DATE-OBS: (from CFITSIO error stack:)
             input date string has illegal format:
             UTC_format
             



## Parameter interface

### Interactive

<font color=green>This should then prompt for the non-hidden fdump parameters, which it does:</font>

In [54]:
result=hsp.fdump()
print(result.stdout)

Name of FITS file and [ext#]> Crab_sum_pha.fits
Name of optional output file> STDOUT
Names of columns> RATE
Lists of rows> 1-2
More?> yes
SIMPLE  =                    T / file does conform to FITS standard
BITPIX  =                    8 / number of bits per data pixel
NAXIS   =                    0 / number of data axes
EXTEND  =                    T / FITS dataset may contain extensions
COMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy
COMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H
COMMENT
COMMENT This file was written by the ISDC Data Access Layer (DAL) version
COMMENT 3.0.2. For Questions or further information contact the ISDC at:
COMMENT
COMMENT      INTEGRAL Science Data Centre
COMMENT      Chemin d'Ecogia 16
COMMENT      CH-1290 Versoix, Switzerland
COMMENT
COMMENT The data structures within make use of the FITS Hierarchical
COMMENT Grouping Convention by Jennings, Pence, Folk and Schlesinger. See
COMMENT The NASA/G

<font color=red>**Why does this prompt differently from the command line:**  It looks like heasoftpy is doing the righ tthing and ftverify on the command line is not.  These params have mode a, and the 'mode' parameter itself is set to 'ql'.  So why does ftverify NOT query for them?</font>

<code>
In [4]: hsp.ftverify()
Name of FITS file to verify > foo.fits
number of errors (output) > -
number of warnings (output) > -
Out[4]: Result(returncode=0, stdout=' \n               ftverify 4.18 (CFITSIO V3.420)               \n               ------------------------------               \n \nHEASARC conventions are being checked.\n \nFile: foo.fits\n*** Error:   (from CFITSIO error stack:)\n             failed to find or open the following file: (ffopen)\n             foo.fits\n             \n**** Abort Verification: Fatal Error. ****\n', stderr=None, params={'infile': 'foo.fits', 'numerrs': '-', 'numwrns': '-'}, custom=None)


Suspended
(hspdev) 9:43 ~/space/sw/heasoft > ftverify foo.fits

               ftverify 4.18 (CFITSIO V3.420)
               ------------------------------

HEASARC conventions are being checked.

File: foo.fits
*** Error:   (from CFITSIO error stack:)
             failed to find or open the following file: (ffopen)
             foo.fits

**** Abort Verification: Fatal Error. ****
</code>

<font color=green>It also returns a params dictionary that you can hand it back again</font>

In [57]:
print(result.params)
result.params['prhead']='no'
print(hsp.fdump(**result.params).stdout)

{'infile': 'Crab_sum_pha.fits', 'outfile': 'STDOUT', 'columns': 'RATE', 'rows': '1-2', 'more': 'yes', 'prhead': 'no'}
 
 
        RATE
        count/s
      1  -2.291760332925691E-01
      2   6.312281554294670E+01



### Learning behavior

<font color=green>This seems to work</font>

In [58]:
## Set up a local pfiles we can mess with
if os.path.exists("pfiles.local"):
    shutil.rmtree("pfiles.local")
os.mkdir("pfiles.local")
os.environ["PFILES"]="./pfiles.local;/Users/tjaffe/space/sw/heasoft/heasoft-6.22.1/x86_64-apple-darwin17.4.0/syspfiles"
os.listdir("pfiles.local")

[]

In [59]:
os.environ['PFILES']

'./pfiles.local;/Users/tjaffe/space/sw/heasoft/heasoft-6.22.1/x86_64-apple-darwin17.4.0/syspfiles'

Nothing in the pfiles.  Call with just the required input parameter:

In [60]:
result=hsp.fdump(infile="Crab_sum_pha.fits",outfile='STDOUT',columns='RATE',rows='1-2',more='yes')
result

Result(returncode=0, stdout="SIMPLE  =                    T / file does conform to FITS standard\nBITPIX  =                    8 / number of bits per data pixel\nNAXIS   =                    0 / number of data axes\nEXTEND  =                    T / FITS dataset may contain extensions\nCOMMENT   FITS (Flexible Image Transport System) format is defined in 'Astronomy\nCOMMENT   and Astrophysics', volume 376, page 359; bibcode: 2001A&A...376..359H\nCOMMENT\nCOMMENT This file was written by the ISDC Data Access Layer (DAL) version\nCOMMENT 3.0.2. For Questions or further information contact the ISDC at:\nCOMMENT\nCOMMENT      INTEGRAL Science Data Centre\nCOMMENT      Chemin d'Ecogia 16\nCOMMENT      CH-1290 Versoix, Switzerland\nCOMMENT\nCOMMENT The data structures within make use of the FITS Hierarchical\nCOMMENT Grouping Convention by Jennings, Pence, Folk and Schlesinger. See\nCOMMENT The NASA/GSFC FITS Users Guide Version 4.0 for more information.\nCOMMENT Cfitsio version: 2.490000\nEN

Now set a parameter to change its behavior and not print the header keywords.  Make sure that this change is picked up.  Currently, the command line is not constructed with all parameters in the pfile, so this works only because the ftool is doing this behind the scenes.  

In [61]:
os.system("pset fdump prhead=no")

0

<font color=green>The above runs correctly using prhead=no</font>

In [62]:
hsp.fdump(**result.params)

Result(returncode=0, stdout=' \n \n        RATE\n        count/s\n      1  -2.291760332925691E-01\n      2   6.312281554294670E+01\n', stderr=None, params={'infile': 'Crab_sum_pha.fits', 'outfile': 'STDOUT', 'columns': 'RATE', 'rows': '1-2', 'more': 'yes'}, custom=None)

<font color=green>Check if it learned columns=RATE parameter</yes>

In [68]:
!pget fdump columns

RATE


The above runs correctly *using* prhead=no, but <font color=red>note that it did **not** include all the parameters, such as the prhead=no, in the returned object params.  I think it should.  This is the record of what parameters were run.  I think it should go get all of them, even the ones the user didn't specify.</font>

<font color=red>Now in batch mode, this should *not* know about prhead=no, but it should know about all the other parameters.  This is a mode invented in the requirements doc.  The resulting output file does not have the header, so it is taking it from the local PFILES, which is not what I want mode=batch to do.</font>

In [63]:
hsp.fdump(infile="Crab_sum_pha.fits",mode="batch",outfile="foo.out",columns='RATE',rows='1-2',more='yes')

Result(returncode=0, stdout='', stderr=None, params={'infile': 'Crab_sum_pha.fits', 'mode': 'batch', 'outfile': 'foo.out', 'columns': 'RATE', 'rows': '1-2', 'more': 'yes'}, custom=None)

<font color=red>Do we want basic parameters checked before the system call, e.g., that an integer doesn't get a string or that an infile exists?  This is **not implemented yet** but should give a python exception different from the HeasoftpyExecutionError, such as HeasoftpyParameterError</font>

In [64]:
hsp.fdump(infile="missing_file.fits",pagewidth="foo",outfile='STDOUT',columns='RATE',rows='1-2',more='yes')

HeasoftpyExecutionError: 
Error running: fdump
returncode: 2
output:
fdump3.5 : could not get PAGEWIDTH parameter
fdump3.5 : Error Status Returned :    2
fdump3.5 : unknown error status

fdump3.5 :  ***** FITSIO Error Stack Dump *****
